In [0]:
PROJECT_PATH = '/home/dobos/project/ga_isochrones/python:' + \
    '/home/dobos/project/ga_pfsspec_all/python:' + \
    '/home/dobos/project/pysynphot'

GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/grid/phoenix/phoenix_HiRes'
FILTER_PATH = '/datascope/subaru/data/pfsspec/subaru/hsc/filters/fHSC-g.txt'

ARMS = [ 'b', 'r', 'mr', 'n' ]
FIT_ARMS = [ 'b' ]

DETECTOR_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/arms/{}.json'
DETECTORMAP_PATH = '/datascope/subaru/data/pfsspec/drp_pfs_data/detectorMap/detectorMap-sim-{}1.fits'
PSF_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/psf/import/{}.2'
SKY_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/noise/import/sky.see/{}/sky.h5'
MOON_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/noise/import/moon/{}/moon.h5'

# Radial Velocity fit

Demo code to perform maximum likelihood analysis of
radial velociy measurements between a spectrum and a
given template, drawn from the BOSZ models.

In [0]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from scipy.interpolate import interp1d
import h5py as h5
from tqdm.notebook import trange, tqdm

In [0]:
plt.rc('font', size=7)

In [0]:
# Allow load project as module
for p in reversed(PROJECT_PATH.split(':')):
    sys.path.insert(0, p)

os.environ['PYTHONPATH'] = PROJECT_PATH.split(':')[0]

In [0]:
os.environ['PYTHONPATH']

In [0]:
if 'debugpy' not in sys.modules:
    import debugpy
    debugpy.listen(("localhost", 5683))

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

## Load spectrum grid

In [0]:
from pfs.ga.pfsspec.core.grid import ArrayGrid
from pfs.ga.pfsspec.stellar.grid import ModelGrid
from pfs.ga.pfsspec.stellar.grid.bosz import Bosz
from pfs.ga.pfsspec.stellar.grid.phoenix import Phoenix

In [0]:
fn = os.path.join(GRID_PATH, 'spectra.h5')
grid = ModelGrid(Phoenix(), ArrayGrid)
grid.preload_arrays = False
grid.load(fn, format='h5')

## Simulated observation

In [0]:
from pfs.ga.pfsspec.core import Filter
from pfs.ga.pfsspec.sim.obsmod.pipelines import StellarModelPipeline
from pfs.ga.pfsspec.core import Physics
from pfs.ga.pfsspec.core.obsmod.psf import GaussPsf, PcaPsf
from pfs.ga.pfsspec.sim.obsmod.detectors import PfsDetector
from pfs.ga.pfsspec.sim.obsmod.detectormaps import PfsDetectorMap
from pfs.ga.pfsspec.sim.obsmod.background import Sky
from pfs.ga.pfsspec.sim.obsmod.background import Moon
from pfs.ga.pfsspec.core.obsmod.snr import QuantileSnr

In [0]:
detector = {}

for arm in ARMS:
    detector[arm] = PfsDetector()
    detector[arm].load_json(DETECTOR_PATH.format(arm))
    detector[arm].map = PfsDetectorMap()
    detector[arm].map.load(DETECTORMAP_PATH.format(arm[0]))

    print(arm, detector[arm].map.default_fiberid)
    print(arm, detector[arm].map.get_wave()[0].shape, detector[arm].map.get_wave()[0][[0, -1]], detector[arm].wave)

In [0]:
gauss_psf = {}
pca_psf = {}
template_psf = {}

for arm in ARMS:
    gauss_psf[arm] = GaussPsf()
    gauss_psf[arm].load(os.path.join(PSF_PATH.format(arm), 'gauss.h5'))

    print(f'mean pixel size for arm {arm}', np.diff(detector[arm].get_wave()[0]).mean())
    print(f'mean sigma and FWHM for arm {arm}', gauss_psf[arm].sigma.mean(), 2.355 * gauss_psf[arm].sigma.mean())

    s = gauss_psf[arm].get_optimal_size(grid.wave)
    print(f'optimal kernel size for arm {arm}:', s)

    pca_psf[arm] = PcaPsf()
    pca_psf[arm].load(os.path.join(PSF_PATH.format(arm), 'pca.h5'))

    template_psf[arm] = PcaPsf.from_psf(gauss_psf[arm], grid.wave, size=s, truncate=5)
    print(grid.wave.shape, 
        template_psf[arm].wave.shape, template_psf[arm].dwave.shape, template_psf[arm].pc.shape)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

for arm in ARMS:
    ax.plot(gauss_psf[arm].wave, gauss_psf[arm].sigma)

ax.set_xlabel(r'$\lambda$ [AA]')
ax.set_ylabel(r'LSF sigma [AA]')

ax.grid()

In [0]:
# for arm in ARMS:
#     f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

#     ### Gauss ###

#     idx = np.digitize(5000, grid.wave)

#     w = grid.wave[idx - s // 2:idx + s // 2 + 1]
#     dw = w - grid.wave[idx]
#     k = gauss_psf[arm].eval_kernel_at(grid.wave[idx], dw, normalize=True)
#     print('k', k.shape)

#     ax.plot(dw, k[0], '.-', ms=0.5, lw=0.5)

#     ###########

#     idx = np.digitize(5000, template_psf[arm].wave)

#     w, dw, kk, _, _ = template_psf[arm].eval_kernel(template_psf[arm].wave)
#     print('dw, kk', dw.shape, kk.shape)

#     ax.plot(dw[idx], kk[idx], '.--', ms=0.5, lw=0.5)

#     ax.set_xlim(-2, 2)
#     ax.set_xlabel(r'$\Delta\lambda$')
#     ax.set_title(f'Comparison of PCA and Gauss PSF for arm {arm}')

#     ax.grid()

In [0]:
# Broad-band filter is common for all arms
filt_hsc_g = Filter()
filt_hsc_g.read(FILTER_PATH)

sky = {}
moon = {}

for arm in ARMS:
    detector_wave, _ = detector[arm].get_wave()
    detector_s = gauss_psf[arm].get_optimal_size(detector_wave)
    print(f'Optimal size of PSF kernel for arm {arm}', detector_s)
    detector[arm].psf = PcaPsf.from_psf(gauss_psf[arm], detector_wave, size=detector_s, truncate=5)

    sky[arm] = Sky()
    sky[arm].load(SKY_PATH.format(arm), format='h5')

    moon[arm] = Moon()
    moon[arm].load(MOON_PATH.format(arm), format='h5')

In [0]:
from pfs.ga.pfsspec.stellar import StellarSpectrum
from pfs.ga.pfsspec.sim.obsmod.observations import PfsObservation
from pfs.ga.pfsspec.sim.obsmod.noise import NormalNoise
from pfs.ga.pfsspec.sim.obsmod.calibration import FluxCalibrationBias
from pfs.ga.pfsspec.core.obsmod.resampling import FluxConservingResampler

In [0]:
obs = {}

for arm in ARMS:
    obs[arm] = PfsObservation()
    obs[arm].detector = detector[arm]
    obs[arm].sky = sky[arm]
    obs[arm].moon = moon[arm]
    obs[arm].noise_model = NormalNoise()

In [0]:
def create_pipeline(arm, grid, calib_bias=False):
    pp = StellarModelPipeline()
    pp.model_res = grid.resolution or 150000
    pp.mag_filter = filt_hsc_g
    pp.observation = obs[arm]
    pp.snr = QuantileSnr(binning=1.0)
    pp.resampler = FluxConservingResampler()
    pp.noise_level = 1.0
    pp.noise_freeze = False
    if calib_bias:
        bias = FluxCalibrationBias(reuse_bias=False)
        bias.amplitude = 0.25
        pp.calibration = bias

    return pp

In [0]:
# def get_observation(arm, rv=0.0, noise_level=1.0, noise_freeze=True, calib_bias=True, mag=19, **kwargs):
#     """
#     Generate a spectrum and calculate the variance (sigma) of realistic observational error.
#     """

#     args = {
#         'mag': mag,
#         'seeing': 0.5,
#         'exp_time': 15 * 60,
#         'exp_count': 4 * 3,
#         'target_zenith_angle': 0,
#         'target_field_angle': 0.0,
#         'moon_zenith_angle': 45,
#         'moon_target_angle': 60,
#         'moon_phase': 0.,
#         'z': Physics.vel_to_z(rv) 
#     }

#     idx = grid.get_nearest_index(**kwargs)

#     spec = grid.get_model_at(idx)

#     pp = StellarModelPipeline()
#     pp.model_res = grid.resolution or 150000
#     pp.mag_filter = filt_hsc_g
#     pp.observation = obs[arm]
#     pp.snr = QuantileSnr(binning=1.0)
#     pp.resampler = FluxConservingResampler()
#     pp.noise_level = noise_level
#     pp.noise_freeze = noise_freeze
#     if calib_bias:
#         bias = FluxCalibrationBias(reuse_bias=False)
#         bias.amplitude = 0.25
#         pp.calibration = bias
#     pp.run(spec, **args)

#     return idx, spec, pp

# def get_template(arm, convolve=True, **kwargs):
#     """
#     Generate a noiseless template spectrum with same line spread function as the
#     observation but keep the original, high-resolution binning.
#     """

#     # TODO: add template caching

#     idx = grid.get_nearest_index(**kwargs)
#     temp = grid.get_model_at(idx)
#     temp.cont = None        # Make sure it's not passed around for better performance
#     temp.mask = None

#     if convolve:
#         temp.convolve_psf(template_psf[arm])

#     return idx, temp


## Verify convolution and resampling

In [0]:
rv = 180.0
M_H = -1.5
T_eff = 4000
log_g = 1.0
a_M = 0.0


for arm in ARMS:
    f, ax = plt.subplots(1, 1, figsize=(6, 2.5), dpi=240)

    idx, spec, pp = get_observation(arm, M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M,
        calib_bias=False, rv=0, noise_freeze=False)
    print(spec.M_H, spec.T_eff, spec.log_g, spec.a_M)
    ax.plot(spec.wave, spec.flux, '.-', ms=1, lw=0.5)

    _, temp = get_template(arm, convolve=False, M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M)
    print(temp.M_H, temp.T_eff, temp.log_g, temp.a_M)
    ax.plot(temp.wave, temp.flux * 1.8e-30, '.-', ms=0.5, lw=0.5)

    _, temp = get_template(arm, convolve=True, M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M)
    print(temp.M_H, temp.T_eff, temp.log_g, temp.a_M)
    ax.plot(temp.wave, temp.flux * 1.8e-30, '.-', ms=0.5, lw=0.5)

    xlimits = {
        'b': (5025, 5060),
        'r': (8498, 8502),
        'mr':
            #(8450, 8570)
            #(8490, 8510)
            (8498, 8502),
        'n': (10030, 10050)
    }

    ax.set_xlim(xlimits[arm])
    #ax.set_ylim(0, 1e-16)

    ax.set_title(f'Convolution test for arm {arm}')
    ax.grid()

# Fit RV

In [0]:
rv = 180.0
M_H = -1.5
T_eff = 4000
log_g = 1.0
a_M = 0.0

spectra = {}
pipelines = {}
templates = {}

for arm in ARMS:
    idx, spec, pp = get_observation(arm, rv=rv, M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M)
    idx, spec1, pp1 = get_observation(arm, rv=rv, M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, noise_level=0, calib_bias=False)
    _, temp = get_template(arm, M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M)
    
    print(spec.M_H, spec.T_eff, spec.log_g, spec.a_M)

    f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

    ax.plot(spec.wave, spec.flux_model, '-', lw=0.3)
    ax.plot(spec.wave, spec.flux_err, '-', lw=0.3)
    ax.set_ylim(0, None)

    ### ###

    f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

    ax.plot(spec.wave, spec.flux, '-', lw=0.3)
    ax.plot(spec.wave, spec.flux_model, '-', lw=0.3)
    ax.plot(spec.wave, spec1.flux, '-', lw=0.3)

    ax.set_title(f'SNR = {spec.snr:.2f}')

    ax.set_xlabel(r'$\lambda$')
    ax.set_ylabel(r'$F\_\lambda$')

    ax.set_xlim(0.99 * spec.wave.min(), 1.01 * spec.wave.max())
    ax.set_ylim(0, np.quantile(spec.flux, 0.99) * 1.2)

    f.tight_layout()

    #ax.set_xlim(8400, 8700)

    ### ###

    f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

    ax.plot(spec.wave, spec.flux / spec1.flux, '-', lw=0.3)
    ax.plot(spec.wave, spec.flux_model / spec1.flux, '-', lw=0.5)

    ax.set_xlabel(r'$\lambda$')
    ax.set_ylabel(r'calibration bias')

    ax.set_xlim(0.99 * spec.wave.min(), 1.01 * spec.wave.max())
    ax.set_ylim(0, np.quantile(spec.flux / spec1.flux, 0.99) * 1.2)

    #ax.set_xlim(8400, 8700)

    spectra[arm] = spec
    pipelines[arm] = pp
    templates[arm] = temp

## Radial velocity

In [0]:
from pfs.ga.pfsspec.stellar.rvfit import RVFit, RVFitTrace

In [0]:
# rv = 180.0
# M_H = -1.5
# T_eff = 3800
# log_g = 1.0
# a_M = 0.0

# dSph RGB star
rv = 180.0
M_H = -1.5,
T_eff = 5000,
log_g = 3.0,
a_M = 0.0,

spectra = {}
pipelines = {}
templates = {}

for arm in ARMS:
    idx, spec, pp = get_observation(arm, 
        calib_bias=False, rv=rv, mag=23,
        M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M)
    _, temp = get_template(arm, convolve=True, M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M)

    spectra[arm] = spec
    pipelines[arm] = pp
    templates[arm] = temp

    print(arm, spec.snr)

In [0]:
FIT_ARMS = ['b', 'mr', 'n']

In [0]:
rvfit = RVFit(trace=RVFitTrace())
rvfit.resampler = FluxConservingResampler()

In [0]:
# Run full fitting
best_rv, best_rv_err = rvfit.fit_rv([spectra[arm] for arm in FIT_ARMS], [templates[arm] for arm in FIT_ARMS], 
    rv_bounds=(150.0, 210.0),
    guess_rv_steps=111)
best_rv, best_rv_err, rvfit.rv0

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(rvfit.trace.guess_rv, rvfit.trace.guess_log_L, '.-', ms=1, lw=0.3)
ax.plot(rvfit.trace.guess_rv, rvfit.trace.guess_fit)
ax.axvline(rv, c='k', label=f'ground truth: {rv:.2f}')
ax.axvline(rvfit.trace.guess_params[1], c='orange', ls='--', label=f'lorentz: {rvfit.trace.guess_params[1]:.2f}')
ax.axvline(best_rv, ls='--', c='r', label=f'best fit: {best_rv:.2f}')

snr = QuantileSnr().get_snr([spectra[arm].flux for arm in FIT_ARMS], [spectra[arm].flux_err for arm in FIT_ARMS])

ax.set_title('mag = {}, SNR = {:.2f}, arms: {}'.format(spectra['mr'].mag, snr / pp.noise_level, ','.join(FIT_ARMS)))
ax.legend()

f.tight_layout()

In [0]:
f, ax = plt.subplots(1, 1, figsize=(7, 2.5), dpi=240)

for arm in ARMS:
    ax.plot(spectra[arm].wave, spectra[arm].flux, '.-', ms=0.3, lw=0.2)

z = Physics.vel_to_z(rv)
print('delta lambda:', 4000 * z)
ax.plot(templates['mr'].wave * (1 + z), templates['b'].flux * 0.8e-32, '-k', lw=0.3)

z = Physics.vel_to_z(best_rv)
print('delta lambda:', 4000 * z)
ax.plot(templates['mr'].wave * (1 + z), templates['b'].flux * 0.8e-32, '-r', lw=0.3)

#ax.set_xlim(3800, 4200)   # b
#ax.set_xlim(8400, 8600)   # mr
ax.set_xlim(10000, 10200)   # n

#ax.set_ylim(0, 2 * np.median(spectra['b'].flux))
#ax.set_ylim(0, 10 * np.median(spectra['b'].flux))

ax.set_ylim(0, 0.5e-17)

ax.grid()

# Run many realizations

In [0]:
from pfs.ga.pfsspec.core.util import SmartParallel
from collections.abc import Iterable

In [0]:
idx, spectra, pipelines = None, None, None

def rvfit_mc_plot(spec, rvfit, rv_gt, rv_fit):
    f, axs = plt.subplots(2, 2, figsize=(3.4, 2.5), dpi=240)

    # Noiseless spectrum

    print(spec.wave.shape)

    axs[0, 0].plot(spec.wave, spec.flux_model, '-', lw=0.3)
    axs[0, 0].plot(spec.wave, spec.flux_err, '-', lw=0.3)
    axs[0, 0].set_ylim(0, None)

    # Noisy spectrum

    axs[0, 1].plot(spec.wave, spec.flux, '-', lw=0.3)
    axs[0, 1].plot(spec.wave, spec.flux_model, '-', lw=0.3)

    axs[0, 1].set_xlabel(r'$\lambda$')
    axs[0, 1].set_ylabel(r'$F_\lambda$')

    axs[0, 1].set_xlim(0.99 * spec.wave.min(), 1.01 * spec.wave.max())
    axs[0, 1].set_ylim(0, np.quantile(spec.flux, 0.99) * 1.2)

    # Calibration bias

    if spec.flux_calibration is not None:
        axs[1, 0].plot(spec.wave, spec.flux_calibration, '-', lw=0.3)

    # RV fit

    axs[1, 1].plot(rvfit.trace.guess_rv, rvfit.trace.guess_log_L)
    axs[1, 1].plot(rvfit.trace.guess_rv, rvfit.trace.guess_fit)
    axs[1, 1].axvline(rv, c='k')
    axs[1, 1].axvline(best_rv, c='r')

    f.suptitle(f'mag = {spec.mag:.2f}, SNR = {spec.snr:.2f}')

    f.tight_layout()

def rvfit_mc_worker(i, rv=180, rv_bounds=(100.0, 300.0), calib_bias=False, mag=22, noise_level=1.0,
        M_H=-0.5, T_eff=5500, log_g=1.0, a_M=0.0):

    global idx, spectra, pipelines
    
    # Sample RV randomly or use a constant value
    if isinstance(rv, Iterable):
        rv_gt = np.random.uniform(*rv)
        print(rv_gt)
        spec = None
    else:
        rv_gt = rv
    
    if spectra is None:
        spectra = {}
        pipelines = {}

        for arm in FIT_ARMS:
            idx, spectra[arm], pipelines[arm] = get_observation(arm, rv=rv_gt, mag=mag, M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M,
                calib_bias=calib_bias, noise_level=noise_level, noise_freeze=False)

        # Reset initial value for optimization so that it will try to guess it from scratch
        rvfit.rv0 = None

    # Copy spectrum before generating the noise to keep original to be reused
    # when rv_gt is constant
    nspectra = {}
    for arm in FIT_ARMS:
        nspec = type(spectra[arm])(orig=spectra[arm])
        nspec.flux_model = nspec.flux
        nspec.apply_noise(pipelines[arm].observation.noise_model, noise_level=noise_level)
        nspectra[arm] = nspec

    snr = QuantileSnr().get_snr([nspectra[arm].flux for arm in FIT_ARMS], [nspectra[arm].flux_err for arm in FIT_ARMS])

    # try:
    rv_fit, rv_err = rvfit.fit_rv([nspectra[arm] for arm in FIT_ARMS], [templates[arm] for arm in FIT_ARMS],
        rv_bounds=rv_bounds)
    # except Exception as ex:
    #     return i, rv_gt, np.nan, np.nan, np.nan

    if False:
        for arm in FIT_ARMS:
            rvfit_mc_plot(nspectra[arm], rvfit, rv_gt, rv_fit)

    return i, rv_gt, rv_fit, rv_err, snr

def rvfit_mc(mc_count=100, mag=22, **kwargs):
    
    """
    Fit RV to many realizations of the same observation. Sample in a range of
    magnitudes.
    """

    global idx, spectra, pipelines

    t = tqdm(total=mc_count)

    idx, spec, pp = None, None, None

    rvfit = RVFit(trace=RVFitTrace())        
    rvfit.resampler = FluxConservingResampler()

    rv_gt = np.empty((mc_count,))
    rv_fit = np.empty((mc_count,))
    rv_err = np.empty((mc_count,))
    rv_snr = np.empty((mc_count,))
    with SmartParallel(verbose=False, parallel=True, threads=12) as p:
        for i, res_rv_gt, res_rv_fit, res_rv_err, res_snr in p.map(rvfit_mc_worker, list(range(mc_count)), mag=mag, **kwargs):
            rv_gt[i] = res_rv_gt
            rv_fit[i] = res_rv_fit
            rv_err[i] = res_rv_err
            rv_snr[i] = res_snr
            t.update(1)

    return rv_gt, rv_fit, rv_err, rv_snr

In [0]:
rv_gt = {}
rv_fit = {}
rv_err = {}
rv_snr = {}

In [0]:
# rv = 180.0
# M_H = -1.5
# T_eff = 4000
# log_g = 1.0
# a_M = 0.0

# dSph RGB star
rv = 180.0
M_H = -1.5,
T_eff = 5000,
log_g = 3.0,
a_M = 0.0,

FIT_ARMS = ['b', 'mr', 'n']
N = 1000

for m in [19, 20, 21, 22, 23]:
    rv_gt[m] = {}
    rv_fit[m] = {}
    rv_err[m] = {}
    rv_snr[m] = {}
    for cb in [False]:    
        rv_gt[m][cb], rv_fit[m][cb], rv_err[m][cb], rv_snr[m][cb] = rvfit_mc(mc_count=N, rv=180, 
            mag=m,            calib_bias=cb)
        print(m, cb, np.sum(np.isnan(rv_fit[m][cb])))

In [0]:
bins = np.linspace(-10, 10, 20)

f, axs = plt.subplots(1, len(rv_fit), figsize=(7, 2), dpi=240)

for i, m in enumerate(rv_fit):
    ax = axs[i]
    for cb in rv_fit[m]:
        hist, _ = np.histogram(rv_fit[m][cb] - rv_gt[m][cb], bins=bins, density=True)
        ax.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid', 
            label="{} flux bias".format('with' if cb else 'no'))

    # ax.axvline(rv, c='r', label="ground truth")

    ax.set_xlabel(r'$\Delta\,$RV [km s$^{-1}]$')
    ax.set_ylabel('')

    ax.set_title(f"mag = {m:.2f}\nsnr = {np.nanmean(rv_snr[m][False]):.2f}")

    ax.grid()

for ax in axs:
    ax.set_ylim(0, 0.5)

for ax in axs[1:]:
    ax.yaxis.set_ticklabels([])

f.suptitle(f"arms: {', '.join(FIT_ARMS)}")

f.tight_layout()

In [0]:
#rv_gt
#rv_fit

rv_bias = {}
rv_std = {}
for m in rv_fit:
    rv_bias[m] = {}
    rv_std[m] = {}
    for cb in rv_fit[m]:
        rv_bias[m][cb] = np.nanmean(rv_fit[m][cb] - rv_gt[m][cb])
        rv_std[m][cb] = np.nanstd(rv_fit[m][cb] - rv_gt[m][cb])


In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot([m for m in rv_std], [rv_std[m][False] for m in rv_std], label='sigma')
ax.plot([m for m in rv_bias], [rv_bias[m][False] for m in rv_bias], label='bias')

ax.set_xlabel('mag g')
ax.set_ylabel(r'$\Delta RV$')
ax.set_title(f"arms: {', '.join(FIT_ARMS)}")
ax.legend()
ax.grid()

In [0]:
import pickle

# with open("rv_fit_22_23_10000.dat", "wb") as f:
#     pickle.dump((rv_fit, rv_gt, rv_err), f)

# with open("rv_fit.dat", "rb") as f:
#    (rv_fit, rv_gt, rv_err) = pickle.load(f)